# Experimenting with AutoML
Explore different AutoML libraries for Scikit-Learn to see whether this can improve our predictions.

### Table of contents
* [Data loading, performance metric & preprocessing](#loaddata)
* [AutoML libraries](#automl)
    * [TPOT](#tpot)
    * [Auto-Sklearn](#autosklearn)
    * [HyperOpt](#hyperopt)
* [Conclusion](#conclusion)

In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer, mean_squared_log_error
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, train_test_split
from tpot import TPOTRegressor
from hpsklearn import HyperoptEstimator, any_regressor, any_preprocessing
from hyperopt import tpe

train_data_file = "../data/train.csv"
test_data_file = "../data/test.csv"
tpot_predictions_file = "../results/02_predictions_tpot.csv"
SEED = 0
CV = 5

## Data loading, performance metric & preprocessing <a class="anchor"  id="loaddata"></a>
The code for loading the data, defining the performance metric and a basic preprocessing is copied from [01_basic.ipynb](jupyter_notebooks/01_basic.ipynb).

In [2]:
# load data
train_df = pd.read_csv(train_data_file)
train_df.set_index("Id", inplace=True)
target_col = "SalePrice"
y_train = train_df[target_col]
X_train = train_df.drop(columns=[target_col])
cat_cols = X_train.select_dtypes(include=["object"]).columns
num_cols = X_train.select_dtypes(exclude=["object"]).columns

X_test = pd.read_csv(test_data_file)
X_test.set_index("Id", inplace=True)

In [3]:
# define performance metric
neg_RMSLE_scorer = make_scorer(
    mean_squared_log_error, greater_is_better=False, squared=False
)


def measure_performance(estimator, X, y, scorer=neg_RMSLE_scorer, cv=CV):
    """Calculate negative RMSLE for train and test set via cross validation."""
    cv_results = cross_validate(
        estimator=estimator,
        X=X,
        y=y,
        cv=cv,
        scoring=scorer,
        return_train_score=True,
    )
    test_error = cv_results["test_score"].mean()
    train_error = cv_results["train_score"].mean()
    return train_error, test_error

In [4]:
# basic preprocessing
simple_imputer = ColumnTransformer(
    transformers=[
        (
            "num_imputer",
            SimpleImputer(strategy="mean", keep_empty_features=True),
            num_cols,
        ),
        (
            "cat_imputer",
            SimpleImputer(strategy="most_frequent", keep_empty_features=True),
            cat_cols,
        ),
    ],
    verbose_feature_names_out=False,
)
simple_imputer.set_output(transform="pandas")

ordinal_encoder = ColumnTransformer(
    transformers=[
        (
            "ordinal_encoder",
            OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1),
            cat_cols,
        )
    ],
    remainder="passthrough",
    verbose_feature_names_out=False,
)
ordinal_encoder.set_output(transform="pandas")

preprocessing = Pipeline(
    steps=[("imputer", simple_imputer), ("encoder", ordinal_encoder)]
)


def safe_log_inverse(log_y):
    # clip values in order to avoid overflows in exponential function
    log_y_clipped = np.clip(log_y, -sys.float_info.max, np.log(sys.float_info.max))
    return np.exp(log_y_clipped)


def log_transform_target(estimator):
    return TransformedTargetRegressor(
        regressor=estimator, func=np.log, inverse_func=safe_log_inverse
    )

In [5]:
def build_pipeline(
    estimator,
    imputer=simple_imputer,
    encoder=ordinal_encoder,
    target_transformer=log_transform_target,
):
    preprocessing = Pipeline(steps=[("imputer", imputer), ("encoder", encoder)])
    pipeline = Pipeline(
        steps=[
            ("preprocessing", preprocessing),
            ("estimator", estimator),
        ]
    )
    pipeline = target_transformer(estimator=pipeline)
    return pipeline

## AutoML libraries <a class="anchor"  id="automl"></a>
AutoML stands for Automated Machine Learning. It is a tool to automatically obtain a machine learning pipeline with a relatively good performance. This is achieved via an optimization of the model selection, hyperparameter tuning and some data preprocessing. There are different applied techniques for optimzation like grid or random search, Bayesian optimization or evolutionary algorithms.


We will test and compare multiple AutoML libraries for scikit-learn: TPOT, Auto-Sklearn and HyperOpt-Sklearn.

### TPOT <a class="anchor"  id="tpot"></a>
TPOT stands for tree based pipeline optimization tool. it uses genetic programming (evolutionary algotihm) for optimization.

The input data must be numerical only. Therefore before running TPOT, missing values are handled and categorical features are encoded.

In [9]:
# silence warning about missing feature names
import warnings

warnings.filterwarnings(
    "ignore", category=UserWarning, message="X does not have valid feature names.*"
)

In [6]:
# run TPOT:
tpot_regr = TPOTRegressor(
    generations=5,
    population_size=50,
    cv=CV,
    scoring=neg_RMSLE_scorer,
    early_stop=3,
    verbosity=2,
    random_state=SEED,
    n_jobs=-1,
)
tpot_pipe = build_pipeline(estimator=tpot_regr)
tpot_pipe.fit(X_train, y_train)
# note: TPOT's output gives the error of the logarithmic target (don't trust this)

                                                                              
Generation 1 - Current best internal CV score: -0.010159967860641663
                                                                              
Generation 2 - Current best internal CV score: -0.009685686888022769
                                                                              
Generation 3 - Current best internal CV score: -0.009685686888022769
                                                                              
Generation 4 - Current best internal CV score: -0.009685686888022769
                                                                              
Generation 5 - Current best internal CV score: -0.009685686888022769
                                                                              
Best pipeline: XGBRegressor(RidgeCV(input_matrix), learning_rate=0.1, max_depth=5, min_child_weight=7, n_estimators=100, n_jobs=1, objective=reg:squarederror, subsample=0.6500000000

TransformedTargetRegressor(func=<ufunc 'log'>,
                           inverse_func=<function safe_log_inverse at 0x000001E2113301F0>,
                           regressor=Pipeline(steps=[('preprocessing',
                                                      Pipeline(steps=[('imputer',
                                                                       ColumnTransformer(transformers=[('num_imputer',
                                                                                                        SimpleImputer(keep_empty_features=True),
                                                                                                        Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'Yea...
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))],
                                                                                         verbose_feature_names_out=False))])),
                                                     ('estimator',
                                                      TPOTRegressor(early_stop=3,
                                                                    generations=5,
                                                                    n_jobs=-1,
                                                                    population_size=50,
                                                                    random_state=0,
                                                                    scoring=make_scorer(mean_squared_log_error, greater_is_better=False, squared=False),
                                                                    verbosity=2))]))

In [12]:
# get pipeline with best TPOT model:
best_tpot_estimator = tpot_pipe.regressor_.named_steps["estimator"].fitted_pipeline_
best_tpot_pipe = build_pipeline(estimator=best_tpot_estimator)

In [10]:
# measure performance:
train_error, test_error = measure_performance(best_tpot_pipe, X_train, y_train)
print(f"Train error: {train_error}; Test error: {test_error}")

Train error: -0.07447309960726847; Test error: -0.12576287036155737


In [11]:
# make test predictions:
best_tpot_pipe.fit(X_train, y_train)
pred_test = best_tpot_pipe.predict(X=X_test)
prediction_df = pd.DataFrame({"Id": X_test.index, "SalePrice": pred_test})
prediction_df.to_csv(tpot_predictions_file, index=False)
prediction_df.head()

,Id,SalePrice
0,1461,123023.882409
1,1462,153238.186450
2,1463,168953.416037
3,1464,192400.710542
4,1465,179411.766092


### Auto-Sklearn <a class="anchor"  id="autosklearn"></a>

### Hyperopt <a class="anchor"  id="hyperopt"></a>

## Conclusion <a class="anchor"  id="conclusion"></a>